**PharmacoDB:** https://pharmacodb.pmgenomics.ca/datasets/1<br>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
from glob import glob
from collections import OrderedDict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Utils
# file_path = os.getcwd()  # os.path.dirname(os.path.relpath(__file__))
# utils_path = os.path.abspath(os.path.join(file_path, 'utils_py'))
# sys.path.append(utils_path)
# import utils_all as utils

import warnings
warnings.filterwarnings('ignore')

SEED=0

In [2]:
DATADIR = '/Users/apartin/Dropbox/work/pilot1/pharmaco/ccle'

In [3]:
files = glob(os.path.join(DATADIR, '*'))
fdict = {os.path.basename(f): pd.read_csv(f, sep='\t') for f in files}
fdict = OrderedDict(sorted(fdict.items(), key=lambda x: x[0]))
list(fdict.keys())

['ccle_cellmeta',
 'ccle_dosedata',
 'ccle_drugmeta',
 'ccle_feameta',
 'ccle_phenometa',
 'ccle_rnaseq',
 'ccle_rspdata',
 'ccle_sensnum']

In [5]:
cellmeta = fdict['ccle_cellmeta']
drugmeta = fdict['ccle_drugmeta']
feameta = fdict['ccle_feameta']; # feameta['EntrezGeneId'].astype('int')
phenometa = fdict['ccle_phenometa']
rnaseq = fdict['ccle_rnaseq']
rspdata = fdict['ccle_rspdata']
sensnum = fdict['ccle_sensnum']
dosemeta = fdict['ccle_dosedata']

In [6]:
print(rnaseq.shape)
rnaseq[:2]

(61958, 929)


,G20460.COR-L24.2,G20461.HSC-3.2,G20462.KMS-11.2,G20463.C2BBe1.2,G20464.COR-L311.2,G20465.HCC-33.2,G20466.5637.2,G20467.COR-L279.2,G20468.DMS_53.2,G20469.JHOS-2.2,...,G41747.OCI-M1.5,G41748.MEC-1.5,G41749.LS513.5,G41750.NCI-H1339.5,G41751.JHH-6.5,G41752.HT115.1,G41753.HT55.1,G41807.HOP-62.1,G41811.EKVX.1,G41821.COLO-680N.1
ENSG00000000003,3.888,4.527,0.067,5.86,2.556,4.388,5.394,3.778,3.931,6.136,...,1.057,0.759,3.739,5.185,3.427,4.008,6.274,4.681,4.585,5.272
ENSG00000000005,0.000,0.000,0.000,0.07,0.000,0.058,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.018,0.000,0.430,0.235,0.000,0.000,0.000


In [7]:
print(phenometa.shape)
phenometa[:2]

(929, 7)


,analysis_id,file_name,size,analysis_data_uri,cells,cellid,batchid
G20502.22Rv1.2,f5605d5d-29dd-4114-818e-145f5053a035,G20502.22Rv1.2.bam,15406648045,https://cghub.ucsc.edu/cghub/data/analysis/dow...,22Rv1,22RV1,NaN
G20505.23132_87.2,9dd014c9-ad96-433f-bba5-09b304c44148,G20505.23132_87.2.bam,11604550534,https://cghub.ucsc.edu/cghub/data/analysis/dow...,23132_87,23132-87,NaN


In [8]:
cell_name_mapping = phenometa['cellid'].to_dict()
rnaseq = rnaseq.rename(columns=cell_name_mapping)
rnaseq = rnaseq[sorted(rnaseq.columns)]

In [9]:
cells = sensnum[sensnum.sum(axis=1) > 0].index.tolist()

In [10]:
len(set(rnaseq.columns.tolist()).intersection(set(cells)))

467

In [14]:
feameta['EntrezGeneId'].isna().sum()

37726

In [17]:
cellmeta.nunique()

cellid                       1061
tissueid                       24
CCLE.cellid                  1053
link                         1046
CCLE.name                    1046
Cell.line.primary.name       1046
Cell.line.aliases             194
Gender                          3
Site.Primary                   24
Histology                      22
Hist.Subtype1                  68
Notes                          54
Source                         10
Expression.arrays            1036
SNP.arrays                    995
Oncomap                         1
Hybrid.Capture.Sequencing       1
dtype: int64

In [16]:
cellmeta.sort_values('Site.Primary')

,cellid,tissueid,CCLE.cellid,link,CCLE.name,Cell.line.primary.name,Cell.line.aliases,Gender,Site.Primary,Histology,Hist.Subtype1,Notes,Source,Expression.arrays,SNP.arrays,Oncomap,Hybrid.Capture.Sequencing
KP-N-RT-BM-1,KP-N-RT-BM-1,autonomic_ganglia,KP-N-RT-BM-1,http://www.broadinstitute.org/ccle/cell%20line...,KPNRTBM1_AUTONOMIC_GANGLIA,KP-N-RT-BM-1,NaN,NaN,autonomic_ganglia,neuroblastoma,NS,NaN,HSRRB,BITTS_p_NCLE_RNA12_HG-U133_Plus_2_D04_698920,PODIA_p_NCLE_DNAAffy7_GenomeWideSNP_6_C05_437510,yes,yes
IMR-32,IMR-32,autonomic_ganglia,IMR-32,http://www.broadinstitute.org/ccle/cell%20line...,IMR32_AUTONOMIC_GANGLIA,IMR-32,NaN,M,autonomic_ganglia,neuroblastoma,NS,NaN,ATCC,GILDS_p_NCLE_RNA11_Redo_HG-U133_Plus_2_G02_587654,LOBBY_p_NCLE_DNAAffy6_GenomeWideSNP_6_C01_437320,yes,yes
KELLY,KELLY,autonomic_ganglia,KELLY,http://www.broadinstitute.org/ccle/cell%20line...,KELLY_AUTONOMIC_GANGLIA,KELLY,NaN,NaN,autonomic_ganglia,neuroblastoma,NS,NaN,DSMZ,NIECE_p_NCLE_RNA3_HG-U133_Plus_2_D09_296078,HONEY_p_NCLE_DNAAffy3_S_GenomeWideSNP_6_C01_29...,yes,yes
CHP-212,CHP-212,autonomic_ganglia,CHP-212,http://www.broadinstitute.org/ccle/cell%20line...,CHP212_AUTONOMIC_GANGLIA,CHP-212,NaN,NaN,autonomic_ganglia,neuroblastoma,NS,NaN,ATCC,NIECE_p_NCLE_RNA3_HG-U133_Plus_2_G10_296152,FASTS_p_NCLE_DNAAffy4_S_GenomeWideSNP_6_D02_33...,yes,yes
KP-N-S19s,KP-N-S19s,autonomic_ganglia,KP-N-SI9s,http://www.broadinstitute.org/ccle/cell%20line...,KPNSI9S_AUTONOMIC_GANGLIA,KP-N-SI9s,KP-N-S19s,M,autonomic_ganglia,neuroblastoma,NS,NaN,HSRRB,BUNDS_p_NCLE_RNA5_HG-U133_Plus_2_B11_419860,FIEFS_p_NCLE_DNA_Affy5_GenomeWideSNP_6_H05_410924,yes,yes
KP-N-YN,KP-N-YN,autonomic_ganglia,KP-N-YN,http://www.broadinstitute.org/ccle/cell%20line...,KPNYN_AUTONOMIC_GANGLIA,KP-N-YN,NaN,NaN,autonomic_ganglia,neuroblastoma,NS,NaN,HSRRB,MAKER_p_NCLE_RNA7_HG-U133_Plus_2_A05_454774,PODIA_p_NCLE_DNAAffy7_GenomeWideSNP_6_B11_437552,yes,yes
MHH-NB-11,MHH-NB-11,autonomic_ganglia,MHH-NB-11,http://www.broadinstitute.org/ccle/cell%20line...,MHHNB11_AUTONOMIC_GANGLIA,MHH-NB-11,NaN,M,autonomic_ganglia,neuroblastoma,NS,NaN,DSMZ,SILOS_p_NCLE_RNA9_HG-U133_Plus_2_A04_523474,AWASH_p_NCLE_DNAAffy10_GenomeWideSNP_6_H08_542886,yes,yes
NB1,NB1,autonomic_ganglia,NB-1,http://www.broadinstitute.org/ccle/cell%20line...,NB1_AUTONOMIC_GANGLIA,NB-1,NaN,M,autonomic_ganglia,neuroblastoma,NS,NaN,HSRRB,WATCH_p_NCLE_RNA8_HG-U133_Plus_2_B04_474582,AWASH_p_NCLE_DNAAffy10_GenomeWideSNP_6_F10_542910,yes,yes
NH-6,NH-6,autonomic_ganglia,NH-6,http://www.broadinstitute.org/ccle/cell%20line...,NH6_AUTONOMIC_GANGLIA,NH-6,NaN,NaN,autonomic_ganglia,neuroblastoma,NS,NaN,HSRRB,AGENT_p_NCLE_RNA6_HG-U133_Plus_2_C06_436538,PODIA_p_NCLE_DNAAffy7_GenomeWideSNP_6_C10_437500,yes,yes
SH-SY5Y,SH-SY5Y,autonomic_ganglia,SH-SY5Y,http://www.broadinstitute.org/ccle/cell%20line...,SHSY5Y_AUTONOMIC_GANGLIA,SH-SY5Y,NaN,F,autonomic_ganglia,neuroblastoma,NS,Identical lines: SH-SY5Y is a subclone derived...,ATCC,BITTS_p_NCLE_RNA12_HG-U133_Plus_2_D01_698752,VAULT_p_NCLE_DNAAffy14_GenomeWideSNP_6_C07_698686,yes,yes
